# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn import ensemble, model_selection, metrics, datasets, tree 

import numpy as np
import pandas as pd

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
data = datasets.load_boston()

In [5]:
X = data['data']
y = data['target']

In [17]:
#разобьем на обучающую и тестовую выборки (оставим последние 25% на тест)
X_train = X[0:380]
X_test = X[380:]
y_train = y[0:380]
y_test = y[380:]

array([10.4, 10.9, 11.3, 12.3,  8.8,  7.2, 10.5,  7.4, 10.2, 11.5, 15.1,
       23.2,  9.7, 13.8, 12.7, 13.1, 12.5,  8.5,  5. ,  6.3,  5.6,  7.2,
       12.1,  8.3,  8.5,  5. , 11.9, 27.9, 17.2, 27.5, 15. , 17.2, 17.9,
       16.3,  7. ,  7.2,  7.5, 10.4,  8.8,  8.4, 16.7, 14.2, 20.8, 13.4,
       11.7,  8.3, 10.2, 10.9, 11. ,  9.5, 14.5, 14.1, 16.1, 14.3, 11.7,
       13.4,  9.6,  8.7,  8.4, 12.8, 10.5, 17.1, 18.4, 15.4, 10.8, 11.8,
       14.9, 12.6, 14.1, 13. , 13.4, 15.2, 16.1, 17.8, 14.9, 14.1, 12.7,
       13.5, 14.9, 20. , 16.4, 17.7, 19.5, 20.2, 21.4, 19.9, 19. , 19.1,
       19.1, 20.1, 19.9, 19.6, 23.2, 29.8, 13.8, 13.3, 16.7, 12. , 14.6,
       21.4, 23. , 23.7, 25. , 21.8, 20.6, 21.2, 19.1, 20.6, 15.2,  7. ,
        8.1, 13.6, 20.1, 21.8, 24.5, 23.1, 19.7, 18.3, 21.2, 17.5, 16.8,
       22.4, 20.6, 23.9, 22. , 11.9])

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

s(i) = -2*(a(x_i) - y_i)

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [25]:
def gbm_predict(X, base_algorithm, coeffs):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithm, coeffs)]) for x in X]

In [26]:
base_algorithm=[]
coeffs=[]
s = y_train 
for i in range(50):
    clf=tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithm.append(clf)
    coeffs.append(0.9)
    s=y_train-gbm_predict(X_train, base_algorithm, coeffs)

In [27]:
rmse=metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithm, coeffs), squared = False)
print(rmse)
with open("grad_boost_2.txt", "w") as fout:
    fout.write(str(rmse))

5.455139376031945


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [28]:
base_algorithm_new=[]
coeffs_new=[]
s = y_train 
for i in range(50):
    clf=tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    clf.fit(X_train, s)
    base_algorithm_new.append(clf)
    coeffs_new.append(0.9/(1.0+i))
    s=y_train-gbm_predict(X_train, base_algorithm_new, coeffs_new)

In [30]:
rmse_new=metrics.mean_squared_error(y_test, gbm_predict(X_test, base_algorithm_new, coeffs_new), squared = False)
print(rmse_new)
with open("grad_boost_3.txt", "w") as fout:
    fout.write(str(rmse_new))

5.241033584774468


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [45]:
for n in [1,5, 10, 20, 50, 100]:
    grboost = ensemble.GradientBoostingRegressor(learning_rate = 0.1, max_depth = n, random_state = 42)
    grboost.fit(X_train, y_train)
    print(grboost.score(X_test, y_test))

-0.06170192934907193
0.23039823173542195
-0.2549617290962507
-0.1652562962038968
-0.1652562962038968
-0.1652562962038968


In [46]:
with open("grad_boost_4.txt", "w") as fout:
    fout.write('2 3')

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [53]:
from sklearn import linear_model

lclf = linear_model.LinearRegression()
lclf.fit(X_train, y_train)
lclf.score(X_test, y_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)